# Full Name: Low Ye Yang, Caleb

## Introduction

Steven is a travel blogger that intends to create a travel food series. He is looking at data from Zomato for inspiration. He wants to find restaurants that have good user ratings and interesting past events. Steven obtained the following data:
- Country-Code.xlsx
- restaurant_data.json

## Importing libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Importing dataset

Taking a look at the 'Country-Code.xlsx' file:

In [3]:
df_country_code = pd.read_excel('Country-Code.xlsx')
df_country_code

,Country Code,Country
0,1,India
1,14,Australia
2,30,Brazil
3,37,Canada
4,94,Indonesia
5,148,New Zealand
6,162,Phillipines
7,166,Qatar
8,184,Singapore
9,189,South Africa


Taking a look at the 'restaurant_data.json' file:

In [4]:
df = pd.read_json('restaurant_data.json')
df.head()

,results_found,results_start,results_shown,restaurants
0,29287,1,20,"[{'restaurant': {'R': {'res_id': 18649486}, 'a..."
1,7625,1,20,"[{'restaurant': {'R': {'res_id': 18707652}, 'a..."
2,21776,1,20,"[{'restaurant': {'R': {'res_id': 18392725}, 'a..."
3,16762,1,20,"[{'restaurant': {'R': {'res_id': 58882}, 'apik..."
4,12026,1,20,"[{'restaurant': {'R': {'res_id': 18893197}, 'a..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   results_found  79 non-null     int64 
 1   results_start  79 non-null     int64 
 2   results_shown  79 non-null     int64 
 3   restaurants    79 non-null     object
dtypes: int64(3), object(1)
memory usage: 2.6+ KB


In [6]:
df.describe()

,results_found,results_start,results_shown
count,79.000000,79.0,79.000000
mean,4495.873418,1.0,16.455696
std,7117.453789,0.0,7.685814
min,0.000000,1.0,0.000000
25%,640.000000,1.0,20.000000
50%,1778.000000,1.0,20.000000
75%,4159.000000,1.0,20.000000
max,29481.000000,1.0,20.000000


At a glance it appears that all columns do not contain any missing values, with `results_found`, `results_start` and `results_shown` containing the number of results found, starting and ending index of results; and `restaurants` containing a semi-structured JSON data of 20 entries of the restaurant data we want. 

However looking at the summary statistics, one abnormality is that there are some rows (min valued) where the `results_found` and `results_shown` are 0. We will explore that in the next section.

## Data Cleaning

Taking a look at the rows where the results_found or results_shown are 0:

In [7]:
df[(df.results_found == 0) | (df.results_shown == 0)]

,results_found,results_start,results_shown,restaurants
16,0,1,0,[]
40,0,1,0,[]
41,0,1,0,[]
42,0,1,0,[]
43,0,1,0,[]
44,0,1,0,[]
45,0,1,0,[]
46,0,1,0,[]
47,0,1,0,[]
48,0,1,0,[]


Upon closer inspection, it appears these rows are empty, and do not contain any data. Hence, they will be filtered out for the rest of this project.

In [8]:
df = df[(df.results_found != 0) & (df.results_shown != 0)]
df.reset_index(drop=True, inplace=True)
df.describe()

,results_found,results_start,results_shown
count,65.000000,65.0,65.0
mean,5464.215385,1.0,20.0
std,7507.710296,0.0,0.0
min,122.000000,1.0,20.0
25%,1184.000000,1.0,20.0
50%,2302.000000,1.0,20.0
75%,4846.000000,1.0,20.0
max,29481.000000,1.0,20.0


### Task 1

We want to extract the following fields, and store the data as restaurants.csv:
- Restaurant Id
- Restaurant Name
- Country
- City
- User Rating Votes
- User Aggregate Rating (in float)
- Cuisines

In [9]:
restaurants_df = df[['restaurants']].explode('restaurants')
restaurants_df = pd.json_normalize(restaurants_df.pop('restaurants'))
restaurants_df.head()

,restaurant.R.res_id,restaurant.apikey,restaurant.id,restaurant.name,restaurant.url,restaurant.location.address,restaurant.location.locality,restaurant.location.city,restaurant.location.city_id,restaurant.location.latitude,...,restaurant.has_table_booking,restaurant.events_url,restaurant.establishment_types,restaurant.medio_provider,restaurant.order_url,restaurant.order_deeplink,restaurant.book_url,restaurant.user_rating.custom_rating_text,restaurant.user_rating.custom_rating_text_background,restaurant.user_rating.rating_tool_tip
0,18649486,cba15beb4c265876a9828f242b4cf41c,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"Unit 1B & 1C, Upper Ground Floor-C, Building 1...","Cyber Hub, DLF Cyber City",Gurgaon,1,28.4936741035,...,0,https://www.zomato.com/ncr/the-drunken-botanis...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,308322,cba15beb4c265876a9828f242b4cf41c,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial?utm_sour...,"9-A & 12, Hauz Khas Village, New Delhi",Hauz Khas Village,New Delhi,1,28.5542935327,...,1,https://www.zomato.com/HauzKhasSocial/events#t...,[],1,https://www.zomato.com/HauzKhasSocial/order?ut...,,https://www.zomato.com/HauzKhasSocial/book?utm...,NaN,NaN,NaN
2,18856789,cba15beb4c265876a9828f242b4cf41c,18856789,AIR- An Ivory Region,https://www.zomato.com/ncr/air-an-ivory-region...,"1/83, Third Floor, Club Road, West Punjabi Bag...",Punjabi Bagh,New Delhi,1,28.5478090000,...,1,https://www.zomato.com/ncr/air-an-ivory-region...,[],,NaN,NaN,https://www.zomato.com/ncr/air-an-ivory-region...,NaN,NaN,NaN
3,307374,cba15beb4c265876a9828f242b4cf41c,307374,AMA Cafe,https://www.zomato.com/ncr/ama-cafe-majnu-ka-t...,"House 6, New Colony, Majnu ka Tila, New Delhi",Majnu ka Tila,New Delhi,1,28.7025817618,...,0,https://www.zomato.com/ncr/ama-cafe-majnu-ka-t...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18238278,cba15beb4c265876a9828f242b4cf41c,18238278,Tamasha,https://www.zomato.com/ncr/tamasha-connaught-p...,"28, Block A, Kasturba Gandhi Marg, Connaught P...",Connaught Place,New Delhi,1,28.6296624581,...,0,https://www.zomato.com/ncr/tamasha-connaught-p...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


Selecting only columns we want:

In [10]:
df1 = restaurants_df[["restaurant.id", "restaurant.name", "restaurant.location.country_id", "restaurant.location.city",
                      "restaurant.user_rating.votes", "restaurant.user_rating.aggregate_rating", "restaurant.cuisines"]]
df1.head()

,restaurant.id,restaurant.name,restaurant.location.country_id,restaurant.location.city,restaurant.user_rating.votes,restaurant.user_rating.aggregate_rating,restaurant.cuisines
0,18649486,The Drunken Botanist,1,Gurgaon,4765,4.4,"Continental, Italian, North Indian, Chinese"
1,308322,Hauz Khas Social,1,New Delhi,13627,4.6,"Continental, American, Asian, North Indian, Ch..."
2,18856789,AIR- An Ivory Region,1,New Delhi,1819,4.1,"North Indian, Chinese, Continental, Asian"
3,307374,AMA Cafe,1,New Delhi,3252,4.4,"Cafe, Juices"
4,18238278,Tamasha,1,New Delhi,8112,4.4,"Finger Food, North Indian, Continental, Italian"


To obtain the country name from the country id, we will merge the current dataframe with the Country Code dataframe provided.

In [11]:
df1 = pd.merge(df1, df_country_code, left_on='restaurant.location.country_id', right_on='Country Code', how='left')
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1300 entries, 0 to 1299
Data columns (total 9 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   restaurant.id                            1300 non-null   object 
 1   restaurant.name                          1300 non-null   object 
 2   restaurant.location.country_id           1300 non-null   int64  
 3   restaurant.location.city                 1300 non-null   object 
 4   restaurant.user_rating.votes             1300 non-null   object 
 5   restaurant.user_rating.aggregate_rating  1300 non-null   object 
 6   restaurant.cuisines                      1300 non-null   object 
 7   Country Code                             1280 non-null   float64
 8   Country                                  1280 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 101.6+ KB


It appears that there are some entries which have null values for `country`, i.e. do not have a valid `country_id`. Taking a look at these entries:

In [12]:
df1[df1.isnull().any(axis=1)]

,restaurant.id,restaurant.name,restaurant.location.country_id,restaurant.location.city,restaurant.user_rating.votes,restaurant.user_rating.aggregate_rating,restaurant.cuisines,Country Code,Country
1140,7210754,The Golconda Bowl,17,Dummy,0,0,"North Indian, Mughlai",NaN,NaN
1141,7204069,California Cantina,17,Dummy,0,0,"Burger, Fast Food, Mexican, American, Pizza",NaN,NaN
1142,7200516,Bardelli's,17,Dummy,0,0,South Indian,NaN,NaN
1143,18133267,Baton Rouge,17,Dummy,0,0,Steak,NaN,NaN
1144,7204711,Dunkin Donuts,17,Dummy,0,0,"Fast Food, Desserts",NaN,NaN
1145,7204029,L'Opera,17,Dummy,0,0,Bakery,NaN,NaN
1146,5,Yoko Sizzlers,17,Dummy,0,0,Japanese,NaN,NaN
1147,7200182,Kauai,17,Dummy,0,0,Healthy Food,NaN,NaN
1148,7204328,Mcdonalds,17,Dummy,0,0,"Italian, North Indian",NaN,NaN
1149,3,Cafe Mangii,17,Dummy,0,0,"Italian, European, Bakery",NaN,NaN


These entries have an dummy valued `country_id` and dummy valued `city`. To clean the data, we will replace the NaN values with "Dummy".

In [13]:
df1['Country'] = df1['Country'].fillna("Dummy")

Renaming the columns, and converting the `User_Rating_Votes` into integers and `User_Aggregate_Rating` to float:

In [14]:
df1 = df1[["restaurant.id", "restaurant.name", "Country", "restaurant.location.city", "restaurant.user_rating.votes", 
           "restaurant.user_rating.aggregate_rating", "restaurant.cuisines"]]
df1.columns = ["Id", "Name", "Country", "City", "User_Rating_Votes", "User_Aggregate_Rating", "Cuisines"]

df1["User_Rating_Votes"] = pd.to_numeric(df1["User_Rating_Votes"])
df1["User_Aggregate_Rating"] = pd.to_numeric(df1["User_Aggregate_Rating"])

df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1300 entries, 0 to 1299
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     1300 non-null   object 
 1   Name                   1300 non-null   object 
 2   Country                1300 non-null   object 
 3   City                   1300 non-null   object 
 4   User_Rating_Votes      1300 non-null   int64  
 5   User_Aggregate_Rating  1300 non-null   float64
 6   Cuisines               1300 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 81.2+ KB


In [15]:
df1.to_csv('restaurants.csv')

### Task 2

We want to extract the list of restaurants that have past event in the month of April 2019 and store the data as restaurant_events.csv.
- Event Id
- Restaurant Id
- Restaurant Name
- Photo URL
- Event Title
- Event Start Date
- Event End Date

*Note: Populate empty values with "NA".*


Selecting the columns we want:

In [16]:
df2 = restaurants_df[["restaurant.id", "restaurant.name", "restaurant.photos_url", "restaurant.zomato_events"]]
df2.head()

,restaurant.id,restaurant.name,restaurant.photos_url,restaurant.zomato_events
0,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,"[{'event': {'event_id': 322331, 'friendly_star..."
1,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial/photos?u...,"[{'event': {'event_id': 332812, 'friendly_star..."
2,18856789,AIR- An Ivory Region,https://www.zomato.com/ncr/air-an-ivory-region...,"[{'event': {'event_id': 336644, 'friendly_star..."
3,307374,AMA Cafe,https://www.zomato.com/ncr/ama-cafe-majnu-ka-t...,NaN
4,18238278,Tamasha,https://www.zomato.com/ncr/tamasha-connaught-p...,NaN


The column `restaurant.zomato_events` contains a semi-structured JSON data of entries of the event data we want. Exploding the data and normalizing:

In [17]:
df2 = df2.explode('restaurant.zomato_events')
df2 = df2.join(pd.json_normalize(df2.pop('restaurant.zomato_events')))
df2.head()

,restaurant.id,restaurant.name,restaurant.photos_url,event.event_id,event.friendly_start_date,event.friendly_end_date,event.friendly_timing_str,event.start_date,event.end_date,event.end_time,...,event.display_date,event.is_end_time_set,event.disclaimer,event.event_category,event.event_category_name,event.book_link,event.types,event.share_data.should_show,event.is_zomato_event,event.url
0,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,322331.0,06 March,28 August,"Wednesday, 6th March - Wednesday, 28th August",2019-03-06,2019-08-28,23:59:59,...,06 March - 28 August,0.0,Restaurants are solely responsible for the ser...,1.0,,,[],0.0,NaN,NaN
1,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial/photos?u...,332812.0,29 March,23 May,"Friday, 29th March - Thursday, 23rd May",2019-03-29,2019-05-23,01:35:00,...,29 March - 23 May,1.0,Restaurants are solely responsible for the ser...,0.0,,,[],0.0,NaN,NaN
2,18856789,AIR- An Ivory Region,https://www.zomato.com/ncr/air-an-ivory-region...,336644.0,10 April,11 April,"Wednesday, 10th April - Thursday, 11th April",2019-04-10,2019-04-11,00:00:00,...,10 April - 11 April,1.0,Restaurants are solely responsible for the ser...,0.0,,,"[{'name': 'Bollywood Night', 'color': '#6454B8'}]",0.0,NaN,NaN
3,307374,AMA Cafe,https://www.zomato.com/ncr/ama-cafe-majnu-ka-t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18238278,Tamasha,https://www.zomato.com/ncr/tamasha-connaught-p...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Selecting the columns we want and filtering event in the month of April 2019. 

*Assumption: An event is considered to have happened in April 2019, as long as there is one day in between the start & end date, in April 2019*

In [18]:
df2 = df2[['event.event_id', 'restaurant.id', 'restaurant.name', 'restaurant.photos_url', 
           'event.title', 'event.start_date', 'event.end_date']]
df2 = df2.loc[((df2['event.start_date'] >= '2019-04-01') & (df2['event.end_date'] <= '2019-04-30')) |
              ((df2['event.start_date'] <= '2019-04-01') & (df2['event.end_date'] >= '2019-04-01')) |
              ((df2['event.start_date'] <= '2019-04-30') & (df2['event.end_date'] >= '2019-04-30'))]
df2.drop_duplicates(inplace=True, ignore_index=True)
df2.columns = ["Event_Id", "Restaurant_Id", "Restaurant_Name", "Photo_Url", 
               "Event_Title", "Event_Start_Date", "Event_End_Date"]

In [19]:
df2

,Event_Id,Restaurant_Id,Restaurant_Name,Photo_Url,Event_Title,Event_Start_Date,Event_End_Date
0,322331.0,18649486,The Drunken Botanist,https://www.zomato.com/ncr/the-drunken-botanis...,BackToBasic Wednesdays !!\n\n\n,2019-03-06,2019-08-28
1,332812.0,308322,Hauz Khas Social,https://www.zomato.com/HauzKhasSocial/photos?u...,Live 20/20 Match Screenings,2019-03-29,2019-05-23
2,336644.0,18856789,AIR- An Ivory Region,https://www.zomato.com/ncr/air-an-ivory-region...,Dhol Bhangra Night,2019-04-10,2019-04-11
3,336889.0,18382360,Local,https://www.zomato.com/ncr/local-connaught-pla...,Cocktail Wednesday,2019-04-10,2019-04-10
4,336888.0,18273624,Cafeteria & Co.,https://www.zomato.com/ncr/cafeteria-co-vijay-...,Cocktail Wednesday,2019-04-10,2019-04-10
...,...,...,...,...,...,...,...
177,305850.0,18136493,Hogshead,https://www.zomato.com/pretoria/hogshead-water...,Smoky Thursdays,2019-03-01,2019-04-30
178,298162.0,18445936,Something's Cooking by J,https://www.zomato.com/pretoria/somethings-coo...,Chocolate Tuesdays,2019-04-05,2019-08-21
179,329300.0,18319871,Zealong Tea Estate,https://www.zomato.com/hamilton/zealong-tea-es...,Review us on Zomato,2019-03-22,2019-04-30
180,329301.0,7903141,Jacks Coffee Lounge,https://www.zomato.com/hamilton/jacks-coffee-l...,Opening Promo,2019-03-22,2019-04-30


In [20]:
df2.to_csv('restaurant_events.csv')

### Task 3

From the dataset (restaurant_data.json), determine the threshold for the different rating text based on aggregate rating. Return aggregates for the following ratings only:
- Excellent
- Very Good
- Good
- Average
- Poor



In [21]:
ratings_df = restaurants_df[["restaurant.user_rating.aggregate_rating","restaurant.user_rating.rating_text"]]
ratings_df.columns = ["User_Aggregate_Rating", "User_Rating_Text"]
ratings_df["User_Aggregate_Rating"] = pd.to_numeric(ratings_df["User_Aggregate_Rating"])

In [23]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   User_Aggregate_Rating  1300 non-null   float64
 1   User_Rating_Text       1300 non-null   object 
dtypes: float64(1), object(1)
memory usage: 20.4+ KB


In [24]:
threshold_df = ratings_df.groupby(['User_Aggregate_Rating', 'User_Rating_Text']).size().reset_index()
threshold_df.columns = ["User_Aggregate_Rating", "User_Rating_Text", "Count"]
threshold_df

,User_Aggregate_Rating,User_Rating_Text,Count
0,0.0,Not rated,23
1,2.2,Poor,1
2,2.5,Average,2
3,2.6,Average,2
4,2.7,Average,1
5,2.8,Average,2
6,2.9,Average,3
7,3.0,Average,1
8,3.1,Average,2
9,3.2,Average,12


Interpretations on the thresholds:
- When `User_Aggregate_Rating` is between 4.5 to 4.9, `User_Rating_Text` will be Excellent
- When `User_Aggregate_Rating` is between 4.0 to 4.4, `User_Rating_Text` will be Very Good
- When `User_Aggregate_Rating` is between 3.5 to 3.9, `User_Rating_Text` will be Good
- When `User_Aggregate_Rating` is between 2.5 to 3.4, `User_Rating_Text` will be Average
- When `User_Aggregate_Rating` is below 2.4, `User_Rating_Text` will be Poor
- Some of the `User_Rating_Text` entries are in languages other than English (For this analysis, we will ignore these entries)

In [25]:
ratings = ["Excellent", "Very Good", "Good", "Average", "Poor"]
rating_text_df = ratings_df.loc[ratings_df['User_Rating_Text'].isin(ratings)]
rating_text_df = rating_text_df.groupby('User_Rating_Text')['User_Aggregate_Rating'].describe()
rating_text_df.sort_values(by=['mean'])

,count,mean,std,min,25%,50%,75%,max
User_Rating_Text,,,,,,,,
Poor,1.0,2.200000,NaN,2.2,2.2,2.2,2.2,2.2
Average,60.0,3.193333,0.237834,2.5,3.2,3.3,3.3,3.4
Good,143.0,3.776224,0.130523,3.5,3.7,3.8,3.9,3.9
Very Good,623.0,4.215891,0.134742,4.0,4.1,4.2,4.3,4.4
Excellent,435.0,4.666207,0.142664,4.5,4.5,4.6,4.8,4.9
